# Application of Benford's Law in Image Forgey Detection

This notebook demonstrates the application of Benford's Law in detecting forged images.

Benford's Law states that in many naturally occurring collections of numbers, the distribution of the first digits follows the logarithmic distribution:

$$F_a = \log_{10}\frac{a + 1}{a}$$

where $a$ is the first digit and $F_a$ is the probability of the first digit being $a$.


In [ ]:
# Plot of Benford's Law distribution

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({"font.size": 11})

# Benford's Law distribution
DIGITS = np.arange(1, 10)
BENFORD = np.log10(1 + 1 / DIGITS)

# Plot
plt.margins(0, tight=True)
plt.plot(DIGITS, BENFORD, "s-", color="black", clip_on=False, markerfacecolor="none")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.title("Benford's Law Distribution")
plt.xlabel("First Digit")
plt.ylabel("Frequency")
plt.ylim(0.001, 0.35)
plt.tick_params(direction="in")

## Dataset

For this notebook, we will be using a dataset that closely matches the one stated in the paper "Benford's law applied to digital forensic analysis" by Fernandes et al (2023):

| Name                            |   Fake   |   Real   |
| ------------------------------- | :------: | :------: |
| Columbia Image Splicing Dataset |   180    |   180    |
| COVERAGE dataset                |   100    |   100    |
| CelebA-HQ dataset               |    -     |   8600   |
| This person does not exist      |   120    |    -     |
| 100K-Faces-HQ Datset            |   8600   |    -     |
| Flickr-Faces-HQ Dataset         |          |   120    |
|                                 |          |          |
| **Total**                       | **9000** | **9000** |

However, due to the paper's lack of clear details on the methodology (for example, the CelebA-HQ dataset consists of 30,000 images, but only 8,600 were used), the dataset recreated is done to the best of my abilities.


In [ ]:
import os

dir_fake = ["../../dataset/fake/100k"]
dir_real = ["../../dataset/real/celeba-hq"]


def gather_files(dirs: list[str]) -> list[str]:
    """Collect all file paths, filter invalid files and return a list of valid file paths."""
    return [
        os.path.join(subdir, file)
        for dir in dirs
        for subdir, _, files in os.walk(dir)
        for file in files
        if os.path.isfile(os.path.join(subdir, file))
    ]


fake_files = gather_files(dir_fake)
real_files = gather_files(dir_real)

print(f"Fake files: {len(fake_files)}, Real files: {len(real_files)}")

Fake files: 9000, Real files: 9000


## Discrete Fourier Transform (DFT)

The methodology is as follows:

1. Read the image and convert it to grayscale.
2. Apply the Discrete Fourier Transform (DFT) to the image to obtain a 2D Power Spectrum.
3. Azimuthally Average the Power Spectrum to obtain a 1D Power Spectrum.
4. Interpolate the 1D Power Spectrum to obtain a 1D Power Spectrum with fixed number of features.
5. Use hypothesis testing to determine if the image is real or fake wrt Benford's Law.


In [ ]:
from functools import partial
from multiprocessing import Pool, cpu_count

import cv2
import scipy.interpolate
from tqdm import tqdm


def azimuthalAverage(magnitude_spectrum: np.ndarray) -> np.ndarray:
    """Calculate the azimuthally averaged 1D power spectrum of an image."""
    height, width = magnitude_spectrum.shape

    # Calculate the indices from the image
    y, x = np.indices([height, width])

    center_y, center_x = (height - 1) / 2, (width - 1) / 2

    r = np.hypot(y - center_y, x - center_x)

    # Get the integer part of the radii (bin size = 1)
    r_int = r.astype(int)

    # Calculate the mean for each radius bin
    tbin = np.bincount(r_int.ravel(), magnitude_spectrum.ravel())
    nr = np.bincount(r_int.ravel())

    radial_prof = tbin / nr

    return radial_prof


def fft(
    filename: str,
    features: int = 1000,
    crop: bool = True,
) -> list[float]:
    """Perform FFT on an image and return the azimuthally averaged 1D power spectrum."""
    try:
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    except Exception as e:
        print(f"Error reading {filename}: {e}")

    height, width = img.shape

    if crop:
        y = height // 3
        x = width // 3
        img = img[y:-y, x:-x]

    # do FFT and shift zero frequency component to center
    frequencies = np.fft.fftshift(np.fft.fft2(img))

    # Calculate the magnitude of the spectrum
    magnitude_spectrum = np.abs(frequencies)

    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = azimuthalAverage(magnitude_spectrum)

    # Interpolate the 1D power spectrum to a fixed number of features
    points = np.linspace(0, features, num=psd1D.size)  # coordinates of points in psd1D
    xi = np.linspace(0, features, num=features)  # coordinates for interpolation

    interpolated = scipy.interpolate.griddata(points, psd1D, xi, method="cubic")

    return interpolated


def multithread_fft(filenames: list[str], **kwargs) -> list[list[float]]:
    with Pool(processes=cpu_count()) as pool:
        results = list(
            tqdm(
                pool.imap(partial(fft, **kwargs), filenames),
                total=len(filenames),
                desc="Performing Feature Extraction",
            )
        )
    return results


In [ ]:
psd1D_total_fake = multithread_fft(fake_files)
psd1D_total_real = multithread_fft(real_files)

# Remove None results if any files failed to process
psd1D_total_fake = [result for result in psd1D_total_fake if result is not None]
psd1D_total_real = [result for result in psd1D_total_real if result is not None]

Performing Feature Extraction: 100%|██████████| 9000/9000 [00:27<00:00, 328.86it/s]


In [ ]:
label_total_fake = np.full(len(psd1D_total_fake), True)
label_total_real = np.full(len(psd1D_total_real), False)

features = psd1D_total_fake + psd1D_total_real
labels = np.concatenate((label_total_fake, label_total_real))

## First Digits Extraction

To prepare our dataset for analysis, we will extract the first digits of the DFT coefficients of the images.


In [ ]:
def get_first_digit(value: float) -> int:
    """Get the first digit of a value."""
    return int(str(abs(value))[0])


def get_first_digit_array(array: list[float]) -> list[int]:
    """Get the first digit of each value in an array."""
    return [get_first_digit(value) for value in array]


def get_digit_counts(array: list[int]) -> list[int]:
    """Count the occurrences of each digit in an array."""
    return [array.count(digit) for digit in DIGITS]


with Pool(processes=cpu_count()) as pool:
    first_digits = list(
        tqdm(
            pool.imap(get_first_digit_array, features),
            total=len(features),
            desc="Getting First Digits",
        )
    )

    first_digits_counts = list(
        tqdm(
            pool.imap(get_digit_counts, first_digits),
            total=len(first_digits),
            desc="Counting First Digits",
        )
    )

Counting First Digits: 100%|██████████| 18000/18000 [00:02<00:00, 8691.98it/s]


## Hypothesis Testing

Here, we will do hypothesis testing to determine whether a picture is considered fake or not. We will use the Pearson's Correlation Coefficient of the distribution of the image's first digits count with respect to the Benford's Law.

If the absolute value of the Pearson's Correlation Coefficient $\rho$ is close to 1, then the first digit distribution follows Benford's Law, and we can consider the image as real. Otherwise, if the absolute value of the Pearson's Correlation Coefficient is not equal to 1, then the first digit distribution does not follow Benford's Law, and we can consider the image as fake.

$$H_0: \rho = 1 \text{(Real Image)}$$
$$H_1: \rho \neq 1 \text{(Fake Image)}$$


In [ ]:
import scipy.stats as stats
import sklearn.metrics as metrics


# H0: The first digits distribution follows Benford's Law
# H1: The first digits distribution does not follow Benford's Law

# Fail to reject H0 when absolute value of correlation coefficient is close to 1
# Reject H0 when absolute value of correlation coefficient is below 1 - alpha

# H0 is equal to false, H1 is equal to true
# So return true when absolute value of correlation coefficient is below 1 - alpha

# (I had to write the above to deconfuse myself)


def test_results(
    threshold: int,
    first_digits_counts: list[list[int]] = first_digits_counts,
) -> dict:
    """Test the goodness of fit for each feature and return the confusion matrix and performance metrics."""
    goodness_of_fit = [
        stats.pearsonr(first_digits_count, BENFORD)
        for first_digits_count in first_digits_counts
    ]

    # calculate True Positive, False Positive, True Negative, False Negative
    results = [abs(coeff) < threshold for coeff, _ in goodness_of_fit]

    # label for fake is 0/False, real is 1/True
    TN, FP, FN, TP = metrics.confusion_matrix(labels, results).ravel()

    return {
        "TN": TN,
        "FP": FP,
        "FN": FN,
        "TP": TP,
        "Precision": metrics.precision_score(labels, results),
        "Recall": metrics.recall_score(labels, results),
        "F1": metrics.f1_score(labels, results),
        "Accuracy": metrics.accuracy_score(labels, results),
    }

In [ ]:
# Print the table using pandas Dataframe
import pandas as pd

THRESHOLD = [0.999, 0.99, 0.95, 0.9, 0.8, 0.75]

with Pool(processes=cpu_count()) as pool:
    results = pool.map(test_results, THRESHOLD)

df = pd.DataFrame.from_records(results, index=THRESHOLD)
df.columns.name = "Threshold"
df

Threshold,TN,FP,FN,TP,Precision,Recall,F1,Accuracy
0.999,1,8999,0,9000,0.500028,1.000000,0.666691,0.500056
0.990,191,8809,12,8988,0.505029,0.998667,0.670821,0.509944
0.950,3888,5112,93,8907,0.635352,0.989667,0.773882,0.710833
0.900,7091,1909,237,8763,0.821121,0.973667,0.890911,0.880778
0.800,8581,419,608,8392,0.952446,0.932444,0.942339,0.942944
0.750,8806,194,935,8065,0.976510,0.896111,0.934585,0.937278
